**Criando conexão no sistema.**

In [ ]:
from google.cloud.dataproc_spark_connect import DataprocSparkSession
from google.cloud.dataproc_v1 import Session
from pyspark.sql import functions as F

# This will create a default Spark session
spark = DataprocSparkSession.builder.getOrCreate()


sc-20251016-143102-i3eqzh Dataproc Session is not active, stopping and creating a new one


████████████████████████████████████████████████████▎                           

**Lendo tabela da bronze**

In [ ]:
df = spark.read.format("bigquery").option("table", "clinic-de.1_bronze.samples").load()

df.printSchema()

root
 |-- timeline: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- event: string (nullable = true)
 |    |    |-- ts: timestamp (nullable = true)
 |-- status: string (nullable = true)
 |-- specimen_type: string (nullable = true)
 |-- patient_id: string (nullable = true)
 |-- test_code: string (nullable = true)
 |-- order_item_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- sample_id: string (nullable = true)



**Selecionando colunas necessarias do dataframe.**

In [ ]:
df_tratado_v1 = (df.select(
    "status",
    "specimen_type",
    "patient_id",
    "test_code",
    "order_item_id",
    "order_id",
    "sample_id"
    )
)

**Verificando se há nulos nas colunas.**

In [ ]:
df_tratado_v1.select([F.sum(F.col(c).isNull().cast("integer")).alias(c) for c in df_tratado_v1.columns]).show()

+---+------+-------------+----------+---------+-------------+--------+---------+
| ts|status|specimen_type|patient_id|test_code|order_item_id|order_id|sample_id|
+---+------+-------------+----------+---------+-------------+--------+---------+
|  0|     0|            0|         0|        0|            0|       0|        0|
+---+------+-------------+----------+---------+-------------+--------+---------+



**Salvando dataframe na silver.**

In [ ]:
(df_tratado_v1.write
        .format("bigquery")
        .option("table", "clinic-de.2_silver.samples")
        .option("writeMethod", "direct")
        .mode("overwrite")
        .save()
    )

print("Tabela salva com sucesso")

Tabela salva com sucesso


In [ ]:
spark.stop()